# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas

import numpy as np
import pandas as pd

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset.

In [73]:
# Run this code:

admissions = pd.read_csv('../Admission_Predict.csv')
#to remove trailing spaces at the end of the column names
admissions.columns = admissions.columns.str.rstrip()

Let's evaluate the dataset by looking at the `head` function.

In [74]:
# Your code here:

admissions.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [75]:
# Your code here:

admissions.isna().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [76]:
# Your code here:

admissions.index = admissions['Serial No.']

Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [77]:
# Your code here:

sum(admissions[['GRE Score', 'CGPA']].duplicated())

0

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

In [78]:
# Your code here:

admissions.set_index(['GRE Score', 'CGPA'], inplace=True)

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [79]:
# Your code here:

admissions.reset_index(inplace=True)

# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [80]:
# Your code here:

sum(admissions.eval('CGPA > 9 & Research == 1'))

101

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [81]:
# Your code here:

np.mean(admissions.query('CGPA > 9 & SOP < 3.5')['Chance of Admit'])

0.8019999999999999

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [82]:
def standardize(col):
    # This function takes a column from a dataframe and returns a standardized column by subtracting the column's mean
    # and dividing by the column's standard deviation.
    
    # Your code here:
    
    return (col - np.mean(col)) / np.std(col)

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [85]:
# Your code here:

admissions['CGPA_std'] = standardize(admissions.CGPA)
admissions['GRE_std'] = standardize(admissions['GRE Score'])
admissions['LOR_std'] = standardize(admissions.LOR)

# You could also use the assign function and do all 3 at once.

We will generate the decision choice at random using the code below. Please run the cell.

In [89]:
# Run this code:

from random import choices
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admissions.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [94]:
# Your code here:

admissions['deciding_column'] =admissions.lookup(admissions.index, decision_choice)

Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [97]:
# Your code here:

admissions['decision'] = np.where(admissions.deciding_column > 0.8, 1, 0)

How many applicants will be accepted to the program using the decision column? Compute the result below.

In [98]:
# Your code here:

admissions.decision.value_counts()

0    294
1     91
Name: decision, dtype: int64

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [108]:
# Your code here:

admissions.columns = (admissions.columns
                            .str.replace(' ', '_')
                            .str.replace('.', '')
                            .str.lower())

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [114]:
# Your code here:

admissions['adjusted_gre'] = pd.cut(np.where(admissions.university_rating >=4, admissions.gre_score + 10, admissions.gre_score), bins=4)